In [1]:
# Save the books

import requests

# Get a request and save it as a txt file

books = {
    "863": "The Mysterious Affair at Styles",
    "1155": "The Secret Adversary",
    "58866": "The Murder on the Links",
    "61168": "The Man in the Brown Suit",
    "61262": "Poirot Investigates",
    "65238": "The Secret of Chimneys",
    "66446": "The Plymouth Express Affair",
    "67160": "The Hunter's Lodge Case",
    "67173": "The Missing Will",
    "69087": "The Murder of Roger Ackroyd",
    "70114": "The Big Four",
    "72824": "The Mystery of the Blue Train"
}

# Go through the dict

for id, title in books.items():
    res = requests.get(f'http://www.gutenberg.org/ebooks/{id}.txt.utf-8')
    # Write to txt file

    cleaned_text = res.text.encode('utf-8', errors='replace').decode('utf-8')
    
    filename = "Data/" + title.replace(" ", "-") + ".txt"
    # with open(filename, 'w', encoding='utf-8') as file:
    #     # Clean res.text to have nothing that cannot be encoded in a .write() call
    #     file.write(cleaned_text)
    #     pass

In [18]:
import nltk
from nltk.tokenize import sent_tokenize

# Manually load the Punkt tokenizer

# Test sentence tokenization
text = "This is a test sentence. Let's see if it works."
print(sent_tokenize(text))

['This is a test sentence.', "Let's see if it works."]


In [19]:
# Python program to generate word vectors using Word2Vec

# importing all necessary modules
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action='ignore')


# Reads file
sample = open("Data/books/The-Mysterious-Affair-at-Styles.txt", encoding="utf-8")
s = sample.read()

# Replaces escape character with space
f = s.replace("\n", " ")

data = []

# iterate through each sentence in the file
for i in sent_tokenize(f):
	temp = []

	# tokenize and normalize the sentence into words
	for j in word_tokenize(i):
		temp.append(j.lower())

	data.append(temp)

# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count=1,
								vector_size=100, window=5)

# Print results
print("Cosine similarity between 'murder' " +
	"and 'investigate' - CBOW : ",
	model1.wv.similarity('murder', 'investigate'))

print("Cosine similarity between 'murder' " +
	"and 'said' - CBOW : ",
	model1.wv.similarity('murder', 'said'))

# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count=1, vector_size=100,
								window=5, sg=1)

# Print results
print("Cosine similarity between 'murder' " +
	"and 'investigate' - Skip Gram : ",
	model2.wv.similarity('murder', 'investigate'))

print("Cosine similarity between 'murder' " +
	"and 'said' - Skip Gram : ",
	model2.wv.similarity('murder', 'said'))


Cosine similarity between 'murder' and 'investigate' - CBOW :  0.9484985
Cosine similarity between 'murder' and 'said' - CBOW :  0.9735845
Cosine similarity between 'murder' and 'investigate' - Skip Gram :  0.9522923
Cosine similarity between 'murder' and 'said' - Skip Gram :  0.8787241


In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize, MWETokenizer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re
from gensim.models import Word2Vec

# Make sure to download resources in separate file outside of Jupyter notebook
# nltk.download('punkt_tab') - pre-trained model for tokenization (sent_tokenize)
# nltk.download('stopwords') - removes common filler words like "the" "and" "is"
# nltk.download('wordnet') - database that groups words into sets of synoynms - used for lemmatization



# Reads file
Styles_file = open("Data/books/The-Mysterious-Affair-at-Styles.txt", encoding="utf-8")
Styles = Styles_file.read()
#Styles = "Testing, this is a test sentence. Hello there my name is Nolan, and today I am tokenizing words!"

# Clean Text - Normalization - Lowercase, Remove Punctuation
Styles = Styles.lower()
#Styles = re.sub(r'[^\w\s]', '', Styles)

# Tokenize
sentences = sent_tokenize(Styles)
tokens = [word_tokenize(sentence) for sentence in sentences]

# Remove Stopwords
stop_words = set(stopwords.words('english'))
tokens = [[word for word in sentence if word not in stop_words] for sentence in tokens]
tokens = [[word for word in sentence if word.isalnum()] for sentence in tokens]

#  Lemmatization 
lemmatizer = WordNetLemmatizer()
# Part of Speech = Verb
#tokens = [[lemmatizer.lemmatize(word, pos='v') for word in sentence] for sentence in tokens]

model = Word2Vec(tokens, vector_size=100, window=5, min_count=1, sg=0) #CBOW model
#model1 = gensim.models.Word2Vec(tokens, min_count=1,
								#vector_size=100, window=5)


#model.build_vocab(tokens)
#model.train(tokens, total_examples=model.corpus_count, epochs=5)



similar_words = model.wv.most_similar('murder', topn=5)
similar_words1 = model.wv.most_similar('poison', topn=5)
poison = model.wv.most_similar('strychnine',topn=5)



#print(tokens)
# #print(sentences)

print(f"Words similar to murder: {similar_words}")
print(f"Words similar to poison: {similar_words1}")
print(f"Words similar to styrchnine: {poison}")


# # PCA 
#word_vectors = model.wv

#words = list(word_vectors.index_to_key)

# word_vecs = [word_vectors[word] for word in words]

# pca = PCA(n_components=2)
# word_vecs_pca = pca.fit_transform(word_vecs)

# plt.figure(figsize=(10,8))
# plt.scatter(word_vecs_pca[:, 0], word_vecs_pca[:, 1])
      
# for i, word in enumerate(words):
#     plt.annotate(word, xy=(word_vecs_pca[i, 0], word_vecs_pca[i, 1]))

# plt.title('PCA of Word Embeddings')
# plt.show()


# ### t-SNE









Words similar to murder: [('said', 0.9131861329078674), ('go', 0.9112963676452637), ('quite', 0.9094844460487366), ('time', 0.9086790680885315), ('poirot', 0.9080378413200378)]
Words similar to poison: [('poirot', 0.9387585520744324), ('would', 0.9386401176452637), ('come', 0.9326707720756531), ('well', 0.9312618374824524), ('quite', 0.9311891794204712)]
Words similar to styrchnine: [('would', 0.9850737452507019), ('inglethorp', 0.984468400478363), ('poirot', 0.9837672114372253), ('dorcas', 0.9811028242111206), ('john', 0.9809287190437317)]
